In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [4]:
!pip install pandasql
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt
from sqlite3 import connect
from scipy.stats.mstats import winsorize, ks_2samp
import seaborn as sns
import pandasql as ps
import scipy as scp

conn = connect(':memory:', timeout=10)
filePath = "/content/gdrive/MyDrive/FinancePaperFIMA"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=8cbfa1581939822c9ebe33d408bc9467759e06acd887d82c363719da6c982b5c
  Stored in directory: /root/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [5]:
file = filePath + "/ccmfund1990to2022.csv"
ccmfund1990to2022 = pd.read_csv(file)
print(ccmfund1990to2022)
print(ccmfund1990to2022.columns)

<ipython-input-5-62969edd8ac4>:2: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccmfund1990to2022 = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [6]:
ccmfund1990to2022.to_sql("ccmfund1990to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY, LPERMNO, datadate, LINKTYPE, LINKENDDT, seq, ceq, at, lt, txditc,
        txdb, itcb, pstkrv, pstkl, pstk, indfmt, datafmt, revt, cogs, xint, xsga
        FROM ccmfund1990to2022
        '''
compustat1990to2022 = pd.read_sql(query, conn)

In [7]:
compustat1990to2022['GVKEY'] = pd.to_numeric(compustat1990to2022['GVKEY'], errors='coerce')
compustat1990to2022['LPERMNO'] = pd.to_numeric(compustat1990to2022['LPERMNO'], errors='coerce')
compustat1990to2022['datadate'] = pd.to_datetime(compustat1990to2022['datadate'], format = '%Y%m%d')
compustat1990to2022['LINKTYPE'] = compustat1990to2022['LINKTYPE'].apply(str)
compustat1990to2022['LINKENDDT'] = pd.to_datetime(compustat1990to2022['LINKENDDT'], format = "%Y%m%d", errors = 'coerce')
compustat1990to2022['seq'] = pd.to_numeric(compustat1990to2022['seq'], errors='coerce')
compustat1990to2022['ceq'] = pd.to_numeric(compustat1990to2022['ceq'], errors='coerce')
compustat1990to2022['at'] = pd.to_numeric(compustat1990to2022['at'], errors='coerce')
compustat1990to2022['lt'] = pd.to_numeric(compustat1990to2022['lt'], errors='coerce')
compustat1990to2022['txditc'] = pd.to_numeric(compustat1990to2022['txditc'], errors='coerce')
compustat1990to2022['txdb'] = pd.to_numeric(compustat1990to2022['txdb'], errors='coerce')
compustat1990to2022['itcb'] = pd.to_numeric(compustat1990to2022['itcb'], errors='coerce')
compustat1990to2022['pstkrv'] = pd.to_numeric(compustat1990to2022['pstkrv'], errors='coerce')
compustat1990to2022['pstkl'] = pd.to_numeric(compustat1990to2022['pstkl'], errors='coerce')
compustat1990to2022['pstk'] = pd.to_numeric(compustat1990to2022['pstk'], errors='coerce')
compustat1990to2022['indfmt'] = compustat1990to2022['indfmt'].apply(str)
compustat1990to2022['datafmt'] = compustat1990to2022['datafmt'].apply(str)
compustat1990to2022['revt'] = pd.to_numeric(compustat1990to2022['revt'], errors='coerce')
compustat1990to2022['cogs'] = pd.to_numeric(compustat1990to2022['cogs'], errors='coerce')
compustat1990to2022['xint'] = pd.to_numeric(compustat1990to2022['xint'], errors='coerce')
compustat1990to2022['xsga'] = pd.to_numeric(compustat1990to2022['xsga'], errors='coerce')



In [8]:
compustat2018to2022 = compustat1990to2022[compustat1990to2022['datadate'] >= "2017-01-01 00:00:00"]

compustat2018to2022 = compustat2018to2022[compustat2018to2022['indfmt'] == "INDL"]
compustat2018to2022 = compustat2018to2022[compustat2018to2022['datafmt'] == "STD"]

compustat2018to2022 = compustat2018to2022[(compustat2018to2022['LINKTYPE']== "LU") | (compustat2018to2022['LINKTYPE'] == "LC")]

compustat2018to2022 = compustat2018to2022[(compustat2018to2022['datadate'] <= compustat2018to2022['LINKENDDT']) | pd.isnull(compustat2018to2022['LINKENDDT'])]

In [9]:
compustat2018to2022.to_sql("compustat2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY as gvkey, LPERMNO as permno, datadate, at,
        COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0) as book_value,
        (revt - cogs - xint - xsga) / (COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0)) as operating_profitability
        FROM compustat2018to2022
        '''
compustat2018to2022 = pd.read_sql(query, conn)
compustat2018to2022['book_value'] = pd.to_numeric(compustat2018to2022['book_value'], errors = 'coerce')
compustat2018to2022['operating_profitability']=pd.to_numeric(compustat2018to2022['operating_profitability'],errors='coerce')

In [10]:
compustat2018to2022['datadate'] = pd.to_datetime(compustat2018to2022['datadate'])
compustat2018to2022['year'] = compustat2018to2022['datadate'].dt.year
compustat2018to2022 = compustat2018to2022.sort_values(by = ['permno', 'datadate'], ascending = True)
compustat2018to2022 = compustat2018to2022.reset_index()
toDrop = []
for i in range(len(compustat2018to2022) - 1):
  if(compustat2018to2022['permno'][i] == compustat2018to2022['permno'][i+1]) and (compustat2018to2022['year'][i] == compustat2018to2022['year'][i+1]):
    toDrop.append(i)
compustat2018to2022.drop(toDrop, inplace = True, axis = 0)
compustat2018to2022.drop('index', inplace = True, axis = 1)

In [11]:
print(compustat2018to2022)
print(compustat2018to2022.columns)

        gvkey  permno   datadate         at  book_value  \
0       12825   10026 2017-09-30    867.228     745.027   
1       12825   10026 2018-09-30    932.013     811.413   
2       12825   10026 2019-09-30   1019.339     895.671   
3       12825   10026 2020-09-30   1056.553     873.911   
4       12825   10026 2021-09-30   1122.219     907.232   
...       ...     ...        ...        ...         ...   
27656  184996   93436 2017-12-31  28655.372    4237.312   
27657  184996   93436 2018-12-31  29739.614    4923.243   
27658  184996   93436 2019-12-31  34309.000    6618.000   
27659  184996   93436 2020-12-31  52148.000   22376.000   
27660  184996   93436 2021-12-31  62131.000   30213.000   

       operating_profitability  year  
0                          NaN  2017  
1                          NaN  2018  
2                          NaN  2019  
3                          NaN  2020  
4                          NaN  2021  
...                        ...   ...  
27656             

In [12]:
compustat2018to2022['investment'] = ""
compustat2018to2022.reset_index(inplace = True, drop = True)
for i in range(1, len(compustat2018to2022)):
  if compustat2018to2022['permno'][i] == compustat2018to2022['permno'][i-1]:
    compustat2018to2022['investment'][i] = compustat2018to2022['at'][i] / compustat2018to2022['at'][i-1]

<ipython-input-12-857b06891b81>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat2018to2022['investment'][i] = compustat2018to2022['at'][i] / compustat2018to2022['at'][i-1]
<ipython-input-12-857b06891b81>:5: RuntimeWarning: invalid value encountered in double_scalars
  compustat2018to2022['investment'][i] = compustat2018to2022['at'][i] / compustat2018to2022['at'][i-1]


In [13]:
compustat2018to2022.replace([np.inf, -np.inf], np.nan, inplace = True)
compustat2018to2022 = compustat2018to2022.dropna()

compustat2018to2022 = compustat2018to2022.reset_index()
compustat2018to2022.drop('index', inplace = True, axis = 1)

In [14]:
compustat2018to2022['year'] = pd.to_numeric(compustat2018to2022['year'], errors = 'coerce')
compustat2018to2022['reference_date'] = compustat2018to2022['year'] + 1
compustat2018to2022['reference_date'] = compustat2018to2022['reference_date'].apply(str)
compustat2018to2022['reference_date'] += "-06-01"
compustat2018to2022['reference_date'] = pd.to_datetime(compustat2018to2022['reference_date'])
compustat2018to2022.drop('year', inplace = True, axis = 1)

print(compustat2018to2022)
print(compustat2018to2022.columns)

        gvkey  permno   datadate         at  book_value  \
0       12096   10028 2017-12-31     13.315       7.756   
1       12096   10028 2018-12-31     13.018       8.413   
2       12096   10028 2019-12-31     27.003      11.194   
3       12096   10028 2020-12-31     40.579      17.578   
4       12096   10028 2021-12-31     59.267      27.627   
...       ...     ...        ...        ...         ...   
18077  184996   93436 2017-12-31  28655.372    4237.312   
18078  184996   93436 2018-12-31  29739.614    4923.243   
18079  184996   93436 2019-12-31  34309.000    6618.000   
18080  184996   93436 2020-12-31  52148.000   22376.000   
18081  184996   93436 2021-12-31  62131.000   30213.000   

       operating_profitability investment reference_date  
0                     0.271532                2018-06-01  
1                     0.240818   0.977694     2019-06-01  
2                     0.306593   2.074282     2020-06-01  
3                     0.392309   1.502759     2021-06-0

In [15]:
file = filePath + "/stocks2018to2022.csv"
stocks2018to2022 = pd.read_csv(file)
print(stocks2018to2022)
print(stocks2018to2022.columns)

        Unnamed: 0  permno                 date exchange       ret  \
0                0   10026  2019-06-28 00:00:00   NASDAQ    0.3730   
1                1   10026  2019-07-31 00:00:00   NASDAQ   15.4644   
2                2   10026  2019-08-30 00:00:00   NASDAQ    3.8851   
3                3   10026  2019-09-30 00:00:00   NASDAQ   -0.2901   
4                4   10026  2019-10-31 00:00:00   NASDAQ   -0.6458   
...            ...     ...                  ...      ...       ...   
115641      115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612   
115642      115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855   
115643      115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609   
115644      115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768   
115645      115645   93436  2022-03-31 00:00:00   NASDAQ -100.0000   

          market_cap       reference_date     gvkey             datadate  \
0       3.030689e+03  2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   
1      

In [16]:
stocks2018to2022.to_sql("stocks2018to2022",conn, if_exists='replace')
compustat2018to2022.to_sql("compustat2018to2022",conn, if_exists='replace')
query='''
        SELECT DISTINCT stocks2018to2022.*,
                        compustat2018to2022.operating_profitability, compustat2018to2022.investment
        FROM stocks2018to2022
        LEFT JOIN compustat2018to2022
        ON stocks2018to2022.permno =compustat2018to2022.permno
        AND stocks2018to2022.reference_date = compustat2018to2022.reference_date
        '''
stocks2018to2022=pd.read_sql(query, conn)
stocks2018to2022.drop('index',inplace=True,axis=1)

In [17]:
print(stocks2018to2022)

        Unnamed: 0  permno                 date exchange       ret  \
0                0   10026  2019-06-28 00:00:00   NASDAQ    0.3730   
1                1   10026  2019-07-31 00:00:00   NASDAQ   15.4644   
2                2   10026  2019-08-30 00:00:00   NASDAQ    3.8851   
3                3   10026  2019-09-30 00:00:00   NASDAQ   -0.2901   
4                4   10026  2019-10-31 00:00:00   NASDAQ   -0.6458   
...            ...     ...                  ...      ...       ...   
115641      115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612   
115642      115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855   
115643      115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609   
115644      115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768   
115645      115645   93436  2022-03-31 00:00:00   NASDAQ -100.0000   

          market_cap       reference_date     gvkey             datadate  \
0       3.030689e+03  2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   
1      

In [18]:
#Robustness sort
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
robustbreakpoints = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6) & (stocks2018to2022['exchange'] == "NYSE")]
robustbreakpoints.to_sql("robustbreakpoints", conn, if_exists = 'replace')
query = '''
       SELECT DISTINCT permno, reference_date, operating_profitability
       FROM robustbreakpoints
       '''
robustbreakpoints = pd.read_sql(query, conn)
robustbreakpoints['reference_date'] = pd.to_datetime(robustbreakpoints['reference_date'])
g = robustbreakpoints.groupby(['reference_date'])
robustsorts = stocks2018to2022[(stocks2018to2022['date'].dt.month == 6)]


robustsorts['value_q30'] = ""
robustsorts['value_q70'] = ""
for i in robustsorts.index:
 robustsorts['value_q30'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.3)
 robustsorts['value_q70'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.7)
robustsorts['robust_portfolio'] = ""
for i in robustsorts.index:
 if robustsorts['operating_profitability'][i] > robustsorts['value_q70'][i]:
   robustsorts['robust_portfolio'][i] = "R"
 elif robustsorts['value_q30'][i] < robustsorts['operating_profitability'][i] <= robustsorts['value_q70'][i]:
   robustsorts['robust_portfolio'][i] = "N"
 else:
   robustsorts['robust_portfolio'][i] = "W"
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
robustsorts.to_sql("robustsorts", conn, if_exists = 'replace')
query = '''
       SELECT DISTINCT stocks2018to2022.*, robustsorts.robust_portfolio
       FROM stocks2018to2022
       LEFT JOIN robustsorts
       ON stocks2018to2022.reference_date = robustsorts.reference_date
       AND stocks2018to2022.permno = robustsorts.permno
       '''
stocks2018to2022 = pd.read_sql(query, conn)
stocks2018to2022.drop('index', inplace = True, axis = 1)


print(stocks2018to2022)
print(stocks2018to2022.columns)

Streaming output truncated to the last 5000 lines.
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsorts['value_q30'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.3)
<ipython-input-18-1d46630df993>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robustsorts['value_q70'][i] = g.get_group(g.get_group(robustsorts['reference_date'][i]).reset_index()['reference_date'][0])['operating_profitability'].quantile(0.7)
<ipython-input-18-1d46630df993>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

        Unnamed: 0  permno                 date exchange       ret  \
0                0   10026  2019-06-28 00:00:00   NASDAQ    0.3730   
1                1   10026  2019-07-31 00:00:00   NASDAQ   15.4644   
2                2   10026  2019-08-30 00:00:00   NASDAQ    3.8851   
3                3   10026  2019-09-30 00:00:00   NASDAQ   -0.2901   
4                4   10026  2019-10-31 00:00:00   NASDAQ   -0.6458   
...            ...     ...                  ...      ...       ...   
115641      115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612   
115642      115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855   
115643      115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609   
115644      115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768   
115645      115645   93436  2022-03-31 00:00:00   NASDAQ -100.0000   

          market_cap       reference_date     gvkey             datadate  \
0       3.030689e+03  2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   
1      

In [19]:
print(stocks2018to2022)
print(stocks2018to2022.columns)

        Unnamed: 0  permno                 date exchange       ret  \
0                0   10026  2019-06-28 00:00:00   NASDAQ    0.3730   
1                1   10026  2019-07-31 00:00:00   NASDAQ   15.4644   
2                2   10026  2019-08-30 00:00:00   NASDAQ    3.8851   
3                3   10026  2019-09-30 00:00:00   NASDAQ   -0.2901   
4                4   10026  2019-10-31 00:00:00   NASDAQ   -0.6458   
...            ...     ...                  ...      ...       ...   
115641      115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612   
115642      115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855   
115643      115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609   
115644      115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768   
115645      115645   93436  2022-03-31 00:00:00   NASDAQ -100.0000   

          market_cap       reference_date     gvkey             datadate  \
0       3.030689e+03  2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   
1      

In [27]:
#Investment sort
stocks2018to2022['date']=pd.to_datetime(stocks2018to2022['date'])
investmentbreakpoints=stocks2018to2022[(stocks2018to2022['date'].dt.month==6)&(stocks2018to2022['exchange']=="NYSE")&(stocks2018to2022['investment']!="")]
investmentbreakpoints.to_sql("investmentbreakpoints",conn, if_exists='replace')
query = '''
        SELECT DISTINCT permno, reference_date, investment
        FROM investmentbreakpoints
        '''
investmentbreakpoints =pd.read_sql(query, conn)
investmentbreakpoints['investment']=pd.to_numeric(investmentbreakpoints['investment'],errors='coerce')
investmentbreakpoints['reference_date']=pd.to_datetime(investmentbreakpoints['reference_date'])
g=investmentbreakpoints.groupby(['reference_date'])

#create new dataframe to sort values
investmentSorts=stocks2018to2022[(stocks2018to2022['date'].dt.month==6)&(stocks2018to2022['investment']!="")]
investmentSorts['investment']=pd.to_numeric(investmentSorts['investment'],errors='coerce')

#calculate new dataframe to sort values
investmentSorts['investment_q30']=""
investmentSorts['investment_q70']=""
for i in investmentSorts.index:
    investmentSorts['investment_q30'][i]=g.get_group(g.get_group(investmentSorts['reference_date'][i]).reset_index()['reference_date'][0])['investment'].quantile(0.3)
    investmentSorts['investment_q70'][i]=g.get_group(g.get_group(investmentSorts['reference_date'][i]).reset_index()['reference_date'][0])['investment'].quantile(0.7)
#sort values into conservatives neutral and aggressive portfolios
investmentSorts['investment_portfolio']=""
for i in investmentSorts.index:
    if investmentSorts['investment'][i]>investmentSorts['investment_q70'][i]:
        investmentSorts['investment_portfolio'][i]="A"
    elif investmentSorts['investment_q30'][i]<investmentSorts['investment'][i]<=investmentSorts['investment_q70'][i]:
        investmentSorts['investment_portfolio'][i]="N"
    else:
        investmentSorts['investment_portfolio'][i]="C"
#left join value portfolio back into stock dataframe
stocks2018to2022.to_sql("stocks2018to2022",conn, if_exists='replace')
investmentSorts.to_sql("investmentSorts",conn, if_exists='replace')
query ='''
        SELECT DISTINCT stocks2018to2022.*, investmentSorts.investment_portfolio
        FROM stocks2018To2022
        LEFT JOIN investmentSorts
        ON stocks2018To2022.reference_date=investmentSorts.reference_date
        AND stocks2018To2022.permno=investmentSorts.permno
        '''
stocks2018to2022=pd.read_sql(query, conn)
stocks2018to2022.drop('index',inplace=True, axis=1)
print(stocks2018to2022)
print(stocks2018to2022.columns)



Streaming output truncated to the last 5000 lines.
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investmentSorts['investment_q30'][i]=g.get_group(g.get_group(investmentSorts['reference_date'][i]).reset_index()['reference_date'][0])['investment'].quantile(0.3)
<ipython-input-27-2148adba71bc>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investmentSorts['investment_q70'][i]=g.get_group(g.get_group(investmentSorts['reference_date'][i]).reset_index()['reference_date'][0])['investment'].quantile(0.7)
<ipython-input-27-2148adba71bc>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in

        Unnamed: 0  permno                 date exchange       ret  \
0                0   10026  2019-06-28 00:00:00   NASDAQ    0.3730   
1                1   10026  2019-07-31 00:00:00   NASDAQ   15.4644   
2                2   10026  2019-08-30 00:00:00   NASDAQ    3.8851   
3                3   10026  2019-09-30 00:00:00   NASDAQ   -0.2901   
4                4   10026  2019-10-31 00:00:00   NASDAQ   -0.6458   
...            ...     ...                  ...      ...       ...   
115641      115641   93436  2021-11-30 00:00:00   NASDAQ    2.7612   
115642      115642   93436  2021-12-31 00:00:00   NASDAQ   -7.6855   
115643      115643   93436  2022-01-31 00:00:00   NASDAQ  -11.3609   
115644      115644   93436  2022-02-28 00:00:00   NASDAQ   -7.0768   
115645      115645   93436  2022-03-31 00:00:00   NASDAQ -100.0000   

          market_cap       reference_date     gvkey             datadate  \
0       3.030689e+03  2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   
1      

In [29]:
stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])
retsizevalue = stocks2018to2022.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retsizerobust = stocks2018to2022.groupby(['date', 'size_portfolio', 'robust_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retsizeinvestment = stocks2018to2022.groupby(['date', 'size_portfolio', 'investment_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retvaluesize = stocks2018to2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retrobustsize = stocks2018to2022.groupby(['date', 'robust_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retinvestmentsize = stocks2018to2022.groupby(['date', 'investment_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))

In [30]:
stocks2018to2022.to_sql("stocks2018to2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date
        FROM stocks2018to2022
        '''
factors = pd.read_sql(query, conn)
factors['date'] = pd.to_datetime(factors['date'])

In [31]:
#SMB portfolio
factors['smb_size'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['smb_size'][i] = np.average(retsizevalue[dateofdata]["S"]) - np.average(retsizevalue[dateofdata]["B"])

factors['smb_robust'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['smb_robust'][i] = np.average(retsizerobust[dateofdata]["S"]) - np.average(retsizerobust[dateofdata]["B"])

factors['smb_investment'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['smb_investment'][i] = np.average(retsizeinvestment[dateofdata]["S"]) - np.average(retsizeinvestment[dateofdata]["B"])

factors['smb'] = ""
for i in factors.index:
  factors['smb'][i] = np.average([factors['smb_size'][i], factors['smb_robust'][i], factors['smb_investment'][i]])

#HML portfolio
factors['hml'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['hml'][i] = np.average(retvaluesize[dateofdata]["H"]) - np.average(retvaluesize[dateofdata]["L"])

#RMW portfolio

factors['rmw'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['rmw'][i] = np.average(retrobustsize[dateofdata]["R"]) - np.average(retrobustsize[dateofdata]["W"])

#CMA portfolio

factors['cma'] = ""
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['cma'][i] = np.average(retinvestmentsize[dateofdata]["C"]) - np.average(retinvestmentsize[dateofdata]["A"])

factors['mkt'] = ""
weightedavgmonthlyret = stocks2018to2022.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights = pd.to_numeric(x.market_cap_weight)))
for i in factors.index:
  dateofdata = factors['date'][i]
  factors['mkt'][i] = weightedavgmonthlyret[dateofdata]

<ipython-input-31-0a15faf46a09>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['smb_size'][i] = np.average(retsizevalue[dateofdata]["S"]) - np.average(retsizevalue[dateofdata]["B"])
<ipython-input-31-0a15faf46a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['smb_robust'][i] = np.average(retsizerobust[dateofdata]["S"]) - np.average(retsizerobust[dateofdata]["B"])
<ipython-input-31-0a15faf46a09>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [32]:
factors['date'] = pd.to_datetime(factors['date'])
factors = factors[factors['date'] < "2022-03-31"]

factors.to_sql("factors", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date, mkt, smb, hml, rmw, cma
        FROM factors
        '''
factors = pd.read_sql(query, conn)

In [33]:
factors['date'] = pd.to_datetime(factors['date'])
factors = factors.sort_values(by = ['date'], ascending = True)

factors = factors.reset_index()
factors.drop('index', inplace = True, axis = 1)

print(factors)
print(factors.columns)

         date        mkt        smb        hml       rmw       cma
0  2019-06-28   7.442855   0.996135   0.434335  2.603622 -0.366370
1  2019-07-31   1.451108  -2.042153   0.066145 -0.649798  0.509209
2  2019-08-30  -2.374261  -3.624964  -4.746700 -0.272303 -0.591295
3  2019-09-30   1.867534   1.160264   5.694920  0.761880  2.578340
4  2019-10-31   2.251724  -0.303682  -2.206578 -0.375657  0.487313
5  2019-11-29   4.040230   0.645772  -2.415970 -0.532038 -0.917238
6  2019-12-31   2.898891   1.703789   2.249337  1.286729  0.902047
7  2020-01-31  -0.200422  -4.455792  -6.096775 -1.094118 -2.428587
8  2020-02-28  -8.209507  -1.061891  -3.377045 -0.884437 -1.900465
9  2020-03-31 -14.038569 -10.308790 -12.349423 -1.851535  0.818011
10 2020-04-30  14.391044   7.703170   2.131972  7.075913 -5.685297
11 2020-05-29   5.059311   2.252691  -5.193029  1.803754 -3.429230
12 2020-06-30   3.238252   4.158727  -2.446018  3.495065 -2.109281
13 2020-07-31   5.810872  -2.553294  -1.637731  1.430012 -2.85

In [75]:
file = filePath + "/FamaFrenchFiveFactorsMonthly.csv"
factorsff = pd.read_csv(file, on_bad_lines='skip')
print(factorsff)
print(factorsff.columns)

       Date   Mkt   SMB   HML   RMW   CMA    RF
0    196307 -0.39 -0.41 -0.97  0.68 -1.18  0.27
1    196308  5.07 -0.80  1.80  0.36 -0.35  0.25
2    196309 -1.57 -0.52  0.13 -0.71  0.29  0.27
3    196310  2.53 -1.39 -0.10  2.80 -2.01  0.29
4    196311 -0.85 -0.88  1.75 -0.51  2.24  0.27
..      ...   ...   ...   ...   ...   ...   ...
706  202205 -0.34 -0.06  8.41  1.44  3.98  0.03
707  202206 -8.43  1.30 -5.97  1.85 -4.70  0.06
708  202207  9.57  1.88 -4.10  0.68 -6.94  0.08
709  202208 -3.78  1.51  0.31 -4.80  1.31  0.19
710  202209 -9.36 -0.96  0.05 -1.40 -0.82  0.19

[711 rows x 7 columns]
Index(['Date', 'Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'RF'], dtype='object')


In [76]:
factorsff.to_sql("factorsff", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT Date as date, Mkt+RF as mkt_ff, SMB as smb_ff, HML as hml_ff, RMW as rmw_ff, CMA as cma_ff
        FROM factorsff
        '''
factorsff = pd.read_sql(query, conn)

In [77]:
factorsff['mkt_ff'] = pd.to_numeric(factorsff['mkt_ff'], errors = 'coerce')
factorsff['smb_ff'] = pd.to_numeric(factorsff['smb_ff'], errors = 'coerce')
factorsff['hml_ff'] = pd.to_numeric(factorsff['hml_ff'], errors = 'coerce')
factorsff['rmw_ff'] = pd.to_numeric(factorsff['rmw_ff'], errors = 'coerce')
factorsff['cma_ff'] = pd.to_numeric(factorsff['cma_ff'], errors = 'coerce')

In [78]:
factors['date']=pd.to_datetime(factors['date'])
factors['month']=factors['date'].dt.month
factorsff['month']=""
for i in factorsff.index:
    factorsff['month'][i]=int(str(factorsff['date'][i])[-2]+str(factorsff['date'][i])[-1]) 


factors['year']=factors['date'].dt.year
factorsff['year']=""
for i in factorsff.index:
    factorsff['year'][i]=int(str(factorsff['date'][i])[0:4]) 


factors.to_sql("factors",conn, if_exists='replace')
factorsff.to_sql("factorsff",conn, if_exists='replace')
query = '''
        SELECT factors.date, factors.mkt, factors.smb, factors.rmw, factors.cma,factors.hml,
            factorsff.mkt_ff, factorsff.smb_ff, factorsff.rmw_ff, factorsff.cma_ff, factorsff.hml_ff
        FROM factors
        LEFT JOIN factorsff
        ON factors.month=factorsff.month
        AND factors.year=factorsff.year
        '''
factors=pd.read_sql(query, conn)
print(factors)

<ipython-input-78-d15ee0be2cf3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factorsff['month'][i]=int(str(factorsff['date'][i])[-2]+str(factorsff['date'][i])[-1])
<ipython-input-78-d15ee0be2cf3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factorsff['year'][i]=int(str(factorsff['date'][i])[0:4])


                   date        mkt        smb       rmw       cma        hml  \
0   2019-06-28 00:00:00   7.442855   0.996135  2.603622 -0.366370   0.434335   
1   2019-07-31 00:00:00   1.451108  -2.042153 -0.649798  0.509209   0.066145   
2   2019-08-30 00:00:00  -2.374261  -3.624964 -0.272303 -0.591295  -4.746700   
3   2019-09-30 00:00:00   1.867534   1.160264  0.761880  2.578340   5.694920   
4   2019-10-31 00:00:00   2.251724  -0.303682 -0.375657  0.487313  -2.206578   
5   2019-11-29 00:00:00   4.040230   0.645772 -0.532038 -0.917238  -2.415970   
6   2019-12-31 00:00:00   2.898891   1.703789  1.286729  0.902047   2.249337   
7   2020-01-31 00:00:00  -0.200422  -4.455792 -1.094118 -2.428587  -6.096775   
8   2020-02-28 00:00:00  -8.209507  -1.061891 -0.884437 -1.900465  -3.377045   
9   2020-03-31 00:00:00 -14.038569 -10.308790 -1.851535  0.818011 -12.349423   
10  2020-04-30 00:00:00  14.391044   7.703170  7.075913 -5.685297   2.131972   
11  2020-05-29 00:00:00   5.059311   2.2

In [79]:
factors.describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9])

,mkt,smb,rmw,cma,hml,mkt_ff,smb_ff,rmw_ff,cma_ff,hml_ff
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,1.762842,0.250914,0.812930,-0.207342,0.177142,1.678182,0.045758,0.657273,0.421515,0.069394
std,5.501896,4.017597,2.016237,2.421247,4.815404,5.380585,3.370821,2.670903,2.513900,5.121531
min,-14.038569,-10.308790,-2.478860,-5.685297,-12.349423,-13.260000,-8.310000,-3.800000,-3.250000,-13.970000
10%,-4.175650,-4.114058,-1.052182,-2.769006,-5.103763,-4.220000,-3.886000,-2.060000,-2.244000,-4.864000
25%,-1.767043,-2.042153,-0.477790,-1.900465,-2.392397,-1.550000,-1.780000,-1.470000,-1.250000,-2.170000
50%,2.826594,-0.262206,-0.042566,-0.277021,-0.061248,2.470000,0.030000,0.300000,-0.430000,-0.720000
75%,4.452327,2.252691,1.930666,0.902047,3.679564,4.640000,1.970000,1.840000,1.740000,3.040000
90%,7.177614,5.989493,3.380682,2.504058,5.703291,7.018000,4.618000,4.788000,3.400000,7.012000
max,14.391044,8.282060,7.075913,7.272665,12.955060,13.650000,7.120000,7.220000,7.710000,12.750000


In [80]:
ks_2samp(factors['mkt'], factors['mkt_ff'])
ks_2samp(factors['smb'], factors['smb_ff'])
ks_2samp(factors['hml'], factors['hml_ff'])

KstestResult(statistic=0.15151515151515152, pvalue=0.851010941578351, statistic_location=-0.16, statistic_sign=-1)

In [81]:
factors.corr(method = 'pearson')

,mkt,smb,rmw,cma,hml,mkt_ff,smb_ff,rmw_ff,cma_ff,hml_ff
mkt,1.000000,0.522495,0.567571,-0.410001,0.148353,0.997743,0.393338,0.241235,-0.219495,0.119082
smb,0.522495,1.000000,0.157703,-0.170370,0.356429,0.520620,0.965295,-0.277588,0.009477,0.362002
rmw,0.567571,0.157703,1.000000,-0.355380,0.105164,0.553150,-0.021136,0.687885,0.014842,0.020858
cma,-0.410001,-0.170370,-0.355380,1.000000,0.642975,-0.435714,-0.059221,0.108269,0.737746,0.680326
hml,0.148353,0.356429,0.105164,0.642975,1.000000,0.116061,0.345756,0.242269,0.602546,0.984622
mkt_ff,0.997743,0.520620,0.553150,-0.435714,0.116061,1.000000,0.396214,0.221436,-0.247596,0.088142
smb_ff,0.393338,0.965295,-0.021136,-0.059221,0.345756,0.396214,1.000000,-0.354256,0.048870,0.377847
rmw_ff,0.241235,-0.277588,0.687885,0.108269,0.242269,0.221436,-0.354256,1.000000,0.112033,0.206436
cma_ff,-0.219495,0.009477,0.014842,0.737746,0.602546,-0.247596,0.048870,0.112033,1.000000,0.613646
hml_ff,0.119082,0.362002,0.020858,0.680326,0.984622,0.088142,0.377847,0.206436,0.613646,1.000000


In [84]:
factors.to_csv('FF5.csv')
!cp FF5.csv "/content/gdrive/MyDrive/FinancePaperFIMA"